<a href="https://colab.research.google.com/github/InhyeokYoo/PyTorch-tutorial-text/blob/master/NLP_FROM_SCRATCH_TRANSLATION_WITH_A_SEQUENCE_TO_SEQUENCE_NETWORK_AND_ATTENTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

> 원본은 [링크](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)를 확인해주세요.

이번시간은 밑바닥부터 "NLP 시작하기(NLP from scratch)"의 세번째이자 마지막 시간입니다. 밑바닥부터 시작하는만큼 우리는 우리만의 클래스와 함수를 직접 만들었습니다. 이번 튜토리얼을 마친후에 바로 따라오는 세 개의 튜토리얼에서 *torchtext*가 이러한 전처리를 어떻게 다루는지를 배우게 될 것입니다.

이번 프로젝트는 뉴럴넷이 불어를 영어로 번역하도록 가르치겠습니다.

```
[KEY: > input, = target, < output]

> il est en train de peindre un tableau .
= he is painting a picture .
< he is painting a picture .

> pourquoi ne pas essayer ce vin delicieux ?
= why not try that delicious wine ?
< why not try that delicious wine ?

> elle n est pas poete mais romanciere .
= she is not a poet but a novelist .
< she not not a poet but a novelist .

> vous etes trop maigre .
= you re too skinny .
< you re all alone .
```

...와 같이 다양한 수준의 성공을 갖게 됩니다.

이는 두 개의 RNN이 함께 작동하여 한 시퀀스에서 다른 시퀀스로 변환하는 [sequence to sequence network](https://arxiv.org/abs/1409.3215)의 간단하지만 강력한 아이디어가 이를 가능케합니다. 인코더는 input sequence를 벡터로 압축하고, 디코더는 이 벡터를 새로운 시퀀스로 펼치게됩니다(unfold).

![](https://pytorch.org/tutorials/_images/seq2seq.png)

이 모델의 성능을 향상시키기 위해, 우리는 [attention mechanism](https://arxiv.org/abs/1409.0473)를 사용할 것입니다. Attention은 디코더로 하여금 input sequence의 특정 범위에 초점을 맞추도록 합니다.

**Recommended Reading:**

I assume you have at least installed PyTorch, know Python, and understand Tensors:

- https://pytorch.org/ For installation instructions
- Deep Learning with PyTorch: A 60 Minute Blitz to get started with PyTorch in general
- Learning PyTorch with Examples for a wide and deep overview
- PyTorch for Former Torch Users if you are former Lua Torch user

It would also be useful to know about Sequence to Sequence networks and how they work:
- Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation
- Sequence to Sequence Learning with Neural Networks
- Neural Machine Translation by Jointly Learning to Align and Translate
- A Neural Conversational Model

You will also find the previous tutorials on NLP From Scratch: Classifying Names with a Character-Level RNN and NLP From Scratch: Generating Names with a Character-Level RNN helpful as those concepts are very similar to the Encoder and Decoder models, respectively.

And for more, read the papers that introduced these topics:
- Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation
- Sequence to Sequence Learning with Neural Networks
- Neural Machine Translation by Jointly Learning to Align and Translate
- A Neural Conversational Model

**Requirements**

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loading data files
이번 프로젝트의 데이터는 불어에서 영어로 번역한 쌍이 수천개가 들어있습니다.

[Open Data Stack Exchange의 질문](https://opendata.stackexchange.com/questions/3888/dataset-of-sentences-translated-into-many-languages)를 통해 open 번역 사이트 https://tatoeba.org/ 를 발견할 수 있었고, https://tatoeba.org/eng/downloads 를 통해 다운로드가 가능합니다. 더 좋은 방법으로는, 누군가가 언어 쌍을 개개의 텍스트 파일로 분리작업 해놓은 추가 작업을 해놓은게 있습니다: https://www.manythings.org/anki/

영어에서 프랑스 쌍은 repo에 포함하기에 너무 크기 때문에, 계속하려면 `data/eng-fra.txt`를 다운받아야 합니다. 이 파일은 tab으로 언어 쌍이 분리되어있습니다.
```
I am cold.    J'ai froid.
```

**Note**
다운로드는 [이곳](https://download.pytorch.org/tutorial/data.zip)에서 받을 수 있습니다.

> 마찬가지로 밑에서 wget으로 받을 수 있게 해놨습니다.

Chraceter-level RNN 튜토리얼과 마찬가지로, 비슷한 character encoding이 사용됩니다. 한 언어에 있는 각 단어를 one-hot vector나, 하나만 제외한(단어의 index) 모든 요소가 0인 거대한 벡터로 표현하겠습니다. 한 언어에 존재하는 글자와 비교하면, 단어는 매우 매우 많이 존재하기 때문에 encoding vector가 매우 큽니다. 그러나 우리는 데이터에 약간의 트릭과 잘라내기를 통해 한 언어당 몇 천개의 단어만 사용할 것입니다.

![](https://pytorch.org/tutorials/_images/word-encoding.png)

추후 네트워크의 input과 target으로 사용하기 위해 각 단어당 고유의 인덱스가 필요합니다. 이러한 과정을 기록하기 위해 `Lang`이라 불리는 helper class를 사용할 것입니다. 이는 word → index (`word2index`)와 index → word  (`index2word`) 딕셔너리를 포함하고, 또한, 희귀한 단어를 처리하기 위한 `word2count`를 포함합니다.


In [ ]:
!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip

--2020-02-04 11:22:53--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 54.192.151.21, 54.192.151.98, 54.192.151.109, ...
Connecting to download.pytorch.org (download.pytorch.org)|54.192.151.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  --.-KB/s    in 0.03s   

2020-02-04 11:22:53 (81.3 MB/s) - ‘data.zip’ saved [2882130/2882130]

Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/I

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:'SOS', 1:'SOS'}
        self.n_words = 2 # 단어의 갯수(SOS와 BOS)

    def add_sentence(self, sentence):
        # sentence내의 단어들을 저장함.
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

이 파일은 모두 Unicode로 저장되어 있습니다. 이를 간단하게 표현하기 위해 Unicode 글자를 ASCII로 표현하고, 구둣점을 잘라내고 소문자로 표현합니다.

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )   # comprehension

# 소문자, 잘라내기(trim), 글자가 아닌 character 잘라내기
def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)   # The backreference \1 (backslash one) references the first capturing group
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

데이터 파일을 읽기 위해 파일을 line으로 split하고, line을 쌍으로 split합니다. 파일은 영어 → 다른 언어로 이루어져있기 때문에 다른 언어 → 영어 번역을 원한다면 `reverse` flag를 추가하여 reverse 쌍을 추가합니다.

In [ ]:
def read_langs(lang1, lang2, reverse=True):
    print("Reading lines...")

    # 파일을 읽고 line으로 분리합니다
    lines = open(f'data/{lang1}-{lang2}.txt', encoding='utf-8').read().strip().split('\n')

    # 모든 line을 pair로 분리하고 normalize합니다.
    pairs = [[normalize_string(s) for s in l.split('\t')] for l in lines]

    # 쌍을 reverse하고, Lang instance를 만듭니다.
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

여기에는 `많은` 예제 문장이 있고 빠르게 학습시키기를 원하기 때문에, 비교적 간편하고 간단하게 문장을 자르겠습니다. 여기서 최고 길이는 10입니다(마지막 구둣점을 포함). 또한, "I am"이나 "He is" 등과 같은 형태의 문장을 필터링합니다.

In [ ]:
MAX_LENGTH = 10 # final

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filter_pair(p):
    # 문장 둘 다 단어수가 10개 이하고, eng_prefixes로 시작하는 문장만 고름.
    return len(p[0].split(' ')) < MAX_LENGTH and \
    len(p[1].split(' ')) < MAX_LENGTH and \
    p[1].startswith(eng_prefixes)

def filter_pairs(pairs):
    return [pair for pair in pairs if filter_pair(pair)]

데이터 전처리를 위한 모든 과정은 다음과 같습니다:
- 텍스트 파일을 읽고 라인으로 분리한 뒤, 라인을 쌍으로 분리
- 텍스트를 노말라이즈하고, 길이와 내용물로 필터링
- 언어쌍의 문장들로 단어 리스트 만들기


In [ ]:
def prepare_data(lang1, lang2, reverse=False):
    # 텍스트 파일을 읽고 이를 라인으로 분리한 뒤, 라인을 쌍으로 분리
    input_lang, output_lang, pairs = read_langs(lang1, lang2, reverse)
    print(f"Read {len(pairs)} sentence pairs")

    # 텍스트를 노말라이즈하고, 길이와 내용물로 필터링
    pairs = filter_pairs(pairs)
    print(f"Trimmed to {len(pairs)} sentence pairs")
    print("Counting words...")
    
    # 언어쌍의 문장들로 단어 리스트 만들기
    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepare_data('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['nous allons faire des courses .', 'we re going shopping .']


# The Seq2Seq Model
RNN은 sequence에 동작하고 다음 단계의 입력으로 자신의 출력을 사용하는 네트워크입니다.

Sequence to sequence network/seq2seq 네트워크/ Encoder Decoder 네트워크는 encoder와 decoder로 불리는 두 개의 RNN으로 이루어진 모델입니다. Encoder는 input sequnce를 읽고 하나의 벡터를 반환합니다. 디코더는 이 벡터를 읽고 output sequence를 반환합니다.

![](https://pytorch.org/tutorials/_images/seq2seq.png)

한 개의 RNN으로 모든 input이 output에 대응하는 sequence를 예측하는 것과는 달리, seq2seq model은 sequnce 길이와 순서에 자유롭습니다. 이는 두 개의 언어 사이를 번역하는 것에 매우 유용합니다.

"Je ne suis pas le chat noir" → "I am not the black cat"와 같은 문장을 고려해봅시다. input 문장의 대부분의 단어는 output 문장의 직역이지만, "chat noir" 과 "black cat"와 같이 순서는 약간 다릅니다. "ne/pas"의 구성 때문에 input sequence에는 하나의 단어가 더 있습니다. 이는 input 단어로부터 직접 올바르게 번역하는 것을 어렵게 만듭니다.

seq2seq에서 encoder는 최상의 시나리오 상 input sequence의 "의미"를 하나의 벡터로 압축합니다. 이는 문장의 N-dimensional space의 한 점을 가리킵니다.

> 더 구체적인 설명

Seq2seq은 품사 판별과 같은 sequential labeling과는 다릅니다. 이는 입력 단어열 [$x_1, x_2, ..., x_n$]의 각 $x_i$에 해당하는 [$y_1, y_2, ..., y_n$]을 출력하는 것으로, 같은 길이에 대해서만 동작합니다.

Seq2seq가 학습하는 기준은 $maximizs \sum P_\theta(y_{1:m}|x_{1:n})$으로, $x_{1:n}$과 $y_{1:m}$의 상관성을 최대화하는 것입니다. 이때, seq2seq은 input sequence의 정보를 하나의 context c에 저장합니다. Encoder RNN의 마지막 hidden state vector는 $c$이고, Decoder는 고정된 context vector $c$와 현재까지 생성된 단어열 $y_{1:i-1}$을 이용하는 language model (sentence generator)입니다.

즉, $P_\theta(y_{1:m}|x_{1:n}) = \Pi_i P_\theta(y_i|y_{1:i-1}, c)$를 통해 학습하는 것입니다. 밑은 이를 도식화한 그림입니다.


![](https://lovit.github.io/assets/figures/seq2seq_fixed_context.png)

## The Encoder
seq2seq의 encoder는 input 문장의 모든 단어들을 위한 하나의 값을 내놓는 RNN입니다. 모든 input word에 대해 encoder는 하나의 벡터와 hidden state를 내놓습니다. 그리고 다음 단어를 위해 hidden state를 사용합니다.

![](https://pytorch.org/tutorials/_images/encoder-network.png)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        # input_size만큼 차원을 받고(즉, embedding할 단어의 개수, lookup table의 차원의 갯수)
        # hidden_size크기의 vector를 반환함
        self.embedding = nn.Embedding(num_embeddings=input_size, embedding_dim=hidden_size)
        self.gru = nn.GRU(input_size=hidden_size, hidden_size=hidden_size)
    
    def forward(self, input, hidden):
        # input: vector <Input_size>
        embedded = self.embedding(input).view(1, 1, -1) # <1 x hidden_size> -> <1 x 1 x hidden_size>
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## The Decoder
디코더는 또 다른 RNN으로, 인코더의 아웃풋 벡터를 받고 단어들의 sequence가 번역을 만들도록 합니다.

### Simple Decoder
가장 단순한 seq2seq 디코더를 위해 우리는 인코더의 마지막 아웃풋만을 사용하겠습니다. 이 마지막 벡터는 때때로 *context vector*라고도 불리며, 말 그대로 전체 시퀀스를 하나의 context로 인코딩합니다. 초기 input token은 start-of-string `<SOS>`이고, 첫 hiddne state는 context vector입니다(인코더의 마지막 output을 의미).

![](https://pytorch.org/tutorials/_images/decoder-network.png)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # 번역 도착 언어의 embedding
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(input_size=hidden_size, hidden_size=hidden_size)
        # output를 내놓는다는 점만 빼면 encoder와 동일
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)   # <1 x hidden_size> ->  <1 x 1 x hidden_size>
        print(f"decoder embedding size:{output.size()}")
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Attention Decoder
만약 context 벡터만이 encoder와 decoder 사이를 지나간다면, 이 벡터는 문장 전체를 인코딩해야 하므로 부담이 많을 것입니다.

Attention은 디코더만의 output의 모든 스텝에서 디코더로 하여금 인코더의 아웃풋의 각기 다른 부분에 "주목"하게 합니다. 가장 먼저 우리는 *attention weight*를 계산하게 됩니다. 이는 인코더의 아웃풋 벡터에 곱해져 weighted combination을 생성할 것입니다. 이 결과물(코드 상에선 `attn_applied`)은 인풋 시퀀스의 특정 부분에 대한 정보를 갖고 있을 것입니다. 따라서 이는 디코더로 하여금 올바른 아웃풋 단어를 선택하게끔 도와줍니다.

![](https://i.imgur.com/1152PYf.png)

어텐션 웨이트를 계산하는 것은 다른 FFN인 `attn`에서 디코더의 인풋과 hidden state를 input으로 이용하여 이루어집니다. 이는 트레이닝 셋에 모든 사이즈의 문장이 있기 때문에, 실제로 만들고 이 레이어를 훈련시키기 위해서는 적용할 수 있는 최대 문장 길이를 선택해야만 합니다(인코더 아웃풋에선 input length). 최대 길이의 문장은 모든 어텐션 웨이트를 사용하는 반면, 짧은 문장은 오로지 몇몇만 사용할 것입니다.

![](https://pytorch.org/tutorials/_images/attention-decoder-network.png)



> 구체적인 설명

Bahdanau et al., (2014) 에서 제시한 바에 따르면 seq2seq와 같이 하나의 문장에 대한 정보를 하나의 context vector $c$로 표현하는 것이 충분하지 않다고 문제를 제기합니다. 이는 Decoder RNN 이 문장을 만들 때 각 단어가 필요한 정보가 다를텐데, sequence to sequence 는 매 시점에 동일한 context $c$를 이용하기 때문입니다. 대신에 $x1, x_2, ..., x_n$에 해당하는 encoder RNN의 hidden state vector $h_1, h_2, ..., h_n$의 조합으로 $y_i$마다 다르게 조합하여 이용하는 방법을 제안합니다.

![](https://lovit.github.io/assets/figures/seq2seq_with_attention.png)

그림처럼 decoder RNN이 $y_i$를 선택할 때 encoder RNN의 $h_j$를 얼마나 사용할지를 $a_{ij}$로 정의합니다. $y_i$의 context vector $c_i$는 $\sum_j a_{ij} \cdot h_j$로 정의되며, $\sum_j a_{ij} =1, a_{ij} >= 0$ 입니다. $a_{ij}$를 attention weight라 하며, 이 역시 neural network에 의하여 학습됩니다.

Weight 는 decoder 의 이전 hidden state $s_i$와 encoder 의 hidden state $h_j$가 input으로 입력되는 feed-forward neural network 입니다. 출력값 $e_{ij}$는 하나의 숫자이며, 이들을 softmax 로 변환하여 확률 형식으로 표현합니다. 그리고 이 확률을 이용하여 encoder hidden vectors 의 weighted average vector 를 만들어 context vector $c_i$로 이용합니다.

$$
a_{ij} = \frac{\exp(e_{ij})}{\sum_j \exp (e_ij)}, e_{ij} = f(s_{i-1}, h_j)
$$

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        # embedding
        embedded = self.embedding(input).view(1, 1, -1) # <1 x 1 x hidden_size>
        embedded = self.dropout(embedded)

        # prev_hidden과 embedded input을 concat하여 attention weight a_ij를 계산
        # embedded[0]: <1 x hidden_size>
        # hidden[0]: <1 x hidden_size>
        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)  # <1 x hidden_size * 2> -> <1 x max_length>
        # encoder_outputs: <max_length(10) x hidden_size>
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))   # -> <1 x 1 x hidden_size>

        output = torch.cat((embedded[0], attn_applied[0]), 1)   # ->  <1 x hidden_size * 2>
        output = self.attn_combine(output).unsqueeze(0) # -> <1 x 1 x hidden_size>

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

**Note**
상대적 위치로 접근하여 길이 제한 문제를 해결한 다른 형태의 attention도 있습니다. [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025) 를 통해 "Local attention"에 대해 읽어보세요.

# Training
## Preparing Training Data
훈련시키기 위해서는 각 언어 쌍에 대해 우리는 인풋 텐서(인풋 문장에 대한 단어의 인덱스)와 타겟 텐서(타겟 문장에 대한 단어의 인덱스)가 필요할 것입니다. 이러한 벡터를 생성하고, EOS 토큰 또한 만들겠습니다.

In [ ]:
def index_from_sentence(lang, sentence):
    # sentence 내 단어를 lang 인스턴스를 통해 index로 변환
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensor_from_sentence(lang, sentence):
    # index_from_sentence를 통해 sentence내 단어의 indices를 얻음
    indexes = index_from_sentence(lang, sentence)
    # EOS 토큰 추가
    indexes.append(EOS_token)
    # 단어 인덱스로 이루어진 (1 x n_words) tensor 반환
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)   # <n_words + 1 x 1>

def tensors_from_pair(pair):
    # 번역 쌍을 tensor로 변환함: <input_length x 1>
    input_tensor = tensor_from_sentence(input_lang, pair[0])
    target_tensor = tensor_from_sentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

## Training the model
학습시키기 위해 우리는 인코더를 통해 인풋 문장을 돌릴 것이고, 마지막 히든 스테이트와 매 순간의 아웃풋에 대해 추적할 것입니다. 그 후, 디코더는 `<SOS>` 토큰을 첫번째 인풋으로 받을 것이고, 인코더의 마지막 히든 스테이트는 디코더의 첫번째 히든 스테이트가 될 것입니다.

*Teacher forcing*은 디코더의 예측을 다음 인풋으로 넣는 대신, 진짜 타겟을 매 다음 인풋으로 사용하는 것에 대한 개념입니다. Teacher forcing을 사용하는 것은 더 빠르게 수렴하도록 하지만, [학습 중인 네트워크가 잘못 사용될 때 불안전 할 수 있습니다.](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf)

tearch forcing된 네트워크의 아웃풋은 일관된 문법을 갖고 있지만 정확한 번역과는 거리가 좀 있습니다. 직관적으로 네트워크는 아웃풋 문법을 표현하는 것을 배우고 한번 교사가 처음 몇 단어를 얘기해주면 의미를 "선택"할 수 있지만 번역에서 처음부터 단어를 생성하는 일은 잘 못합니다.

PyTorch의 autograd가 우리에게 주는 자유로움 덕분에, 간단한 if문만으로 teacher forcing을 사용할지 말지 결정할 수 있습니다.





In [ ]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.init_hidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # 디코더에선 문장의 최대 길이를 정하여 계산
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)   # <10 x 256>

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: 다음 인풋에 target을 feed
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            # BPTT를 위해 더해만 줌.
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: prediction을 다음 인풋에 feed
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            # prediction을 뽑아냄.
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

이는 helper function으로 시간 경과와 진행도를 보여주는 함수입니다.

In [ ]:
import time
import math

def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))

전체 프로세스는 다음 모양과 같습니다.
- 타이머 시작
- optimizer와 criterion 초기화
- 학습 쌍 만들기
- 빈 loss array를 만들어 plotting하기

그 후 `train`을 많이 부르고 때때로 진척도(예제의 %, 걸린 시간, 남은 시간)와 average loss를 출력하겠습니다.

In [ ]:
def train_iters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensors_from_pair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (time_since(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    show_plot(plot_losses)

## Plotting results
Plotting은 matplotlib으로 진행되며, loss 값이 담긴 array를 `plot losses`를 이용합니다.

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def show_plot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

# Evaluation
평가는 대부분 학습과정과 똑같습니다만 타겟이 없으므로 디코더의 결과값을 다시 인풋으로 넣어줘야 합니다. 매 순간 단어를 예측하고, 이를 output string에 추가하며, EOS 토큰이 나올 경우 멈추면 됩니다. 나중에 이를 그려보기 위해 디코더의 어텐션값을 저장해놓겠습니다.



In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensor_from_sentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.init_hidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)   # <10 x 256>
        
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        # 위의 Without Teacher Forcing 부분과 비슷함.
        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

이제 임의의 문장으로 평가하면 됩니다.

In [ ]:
def evaluate_randomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

# Training and Evaluating
이러한 헬퍼 함수를 이용해서(이는 추가적인 작업처럼 보이지만, 여러개의 실험을 더 쉽게 실행할 수 있게끔 도와줍니다) 네트워크를 초기화하고 학습을 시작합니다.

인풋 문장은 까다롭게 필터링 됐다는 점을 생각해보면 이렇게 작은 데이터셋으로는 상대적으로 작은 256의 히든 스테이트를 갖는 네트워크와 하나의 GRU 레이어를 사용합니다. 맥북 CPU 40분 정도 지나면, 꽤나 그럴듯한 결과물을 볼 수 있을겁니다.

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

train_iters(encoder1, attn_decoder1, 75000, print_every=5000)

encoder's output size: torch.Size([1, 1, 256])
encoder's output size: torch.Size([1, 1, 256])
encoder's output size: torch.Size([1, 1, 256])
encoder's output size: torch.Size([1, 1, 256])
encoder's output size: torch.Size([1, 1, 256])
encoder's output size: torch.Size([1, 1, 256])
encoder's output size: torch.Size([1, 1, 256])
attn last output size: torch.Size([1, 1, 256])
attn last output size: torch.Size([1, 1, 256])
attn last output size: torch.Size([1, 1, 256])
attn last output size: torch.Size([1, 1, 256])
attn last output size: torch.Size([1, 1, 256])
attn last output size: torch.Size([1, 1, 256])
attn last output size: torch.Size([1, 1, 256])
encoder's output size: torch.Size([1, 1, 256])
encoder's output size: torch.Size([1, 1, 256])
encoder's output size: torch.Size([1, 1, 256])
encoder's output size: torch.Size([1, 1, 256])
encoder's output size: torch.Size([1, 1, 256])
encoder's output size: torch.Size([1, 1, 256])
attn last output size: torch.Size([1, 1, 256])
attn last out

KeyboardInterrupt: ignored

In [ ]:
evaluate_randomly(encoder1, attn_decoder1)

# Visualizing Attention
어텐션 메카니즘의 유용한 성질은 매우 해석하기 쉬운 결과물을 내놓은단 겁니다. 이는 인풋 시퀀스의 특정 인코더 아웃풋에 가중치를 두어 네트워크가 매 시간에서 어디에 집중하는지 알 수 있습니다.

그냥 `plt.matshow(attentions)`을 실행해 컬럼은 input step, 로우는 output step으로하는 어텐션의 매트릭스로 만들 수 있습니다.

In [ ]:
output_words, attentions = evaluate(encoder1, attn_decoder1, "je suis trop froid .")
plt.matshow(attentions.numpy())

더 나은 비쥬얼라이징을 위하여 축과 레이블을 추가하는 작업을 진행하겠습니다

In [ ]:
def show_attention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') + ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluate_and_show_attention(input_sentence):
    output_words, attentions = evaluate(encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluate_and_show_attention("elle a cinq ans de moins que moi .")

evaluate_and_show_attention("elle est trop petit .")

evaluate_and_show_attention("je ne crains pas de mourir .")

evaluate_and_show_attention("c est un jeune directeur plein de talent .")

# Exercises
- Try with a different dataset
 - Another language pair
 - Human → Machine (e.g. IOT commands)
 - Chat → Response
 - Question → Answer
- Replace the embeddings with pre-trained word embeddings such as word2vec or GloVe
- Try with more layers, more hidden units, and more sentences. Compare the training time and results.
- If you use a translation file where pairs have two of the same phrase (I am test \t I am test), you can use this as an autoencoder. Try this:
 - Train as an autoencoder
 - Save only the Encoder network
 - Train a new Decoder for translation from there